In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

import torch
from Models import *
from functions import *

In [2]:
x = np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', skip_header=1, usecols=range(1, 17), dtype=float)
y = np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', skip_header=1, usecols=(1,2),  dtype=float)

In [3]:
print(x.shape)
print(y.shape)
print(x[:,0].shape)
print(y[:,0].shape)

(6510, 16)
(6510, 2)
(6510,)
(6510,)


In [4]:
#add random column to x
x = np.concatenate((x, np.random.rand(x.shape[0], 1)), axis=1)

In [5]:
import xgboost as xgb

# Train your XGBoost model
model = xgb.XGBRegressor()
 
model.fit(x, y)

# Get feature importances
importances = model.feature_importances_

In [6]:
#show the feature importance
print(importances)

[0.01864266 0.0246649  0.04652443 0.03968611 0.04075368 0.03757898
 0.03857988 0.04536691 0.03735694 0.05118002 0.04490478 0.04416765
 0.11330736 0.11760578 0.03481582 0.22792965 0.03693444]


In [ ]:
#calculate nan values
print(np.isnan(x).sum())
print(np.isnan(y).sum())
#print index of nan values
print(np.argwhere(np.isnan(x)))
print(np.argwhere(np.isnan(y)))

In [ ]:
#preparing train and test loaders

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

In [ ]:
input_size = x.shape[1]
hidden_size = 128
output_size = y.shape[1]
print(input_size)
print(output_size)

model = MLP(input_size, hidden_size, output_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 400

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    
    # Iterate over batches in the training data
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss every few epochs
    if (epoch) % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # Optionally, evaluate the model on the test set after each epoch
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    # with torch.no_grad():
        # for inputs, targets in test_loader:
